In [1]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
constant_patch = '11.22.1'

# 라이엇 개발자 페이지에서 최신 버전 한국 서버 챔피언 정보 가져오기
champion_info = requests.get("http://ddragon.leagueoflegends.com/cdn/" + constant_patch + "/data/ko_KR/champion.json").json()     

In [3]:
# 챔피언 정보 => DataFrame으로 변환
# champion_df는 'key'로 id값, 'name'으로 챔피언 한국어 이름을 가지는 156rows짜리 DataFrame
champion_df = pd.DataFrame(champion_info['data']).T[['key', 'name']]

# champion_df의 'key' column string => numeric 변환
champion_df['key'] = pd.to_numeric(champion_df['key'])

In [4]:
df = pd.read_csv('BIPA_data.csv', index_col=0)

In [5]:
df = df.merge(champion_df, how = 'left', left_on = 'championId', right_on = 'key')

In [6]:
data = pd.DataFrame()
data = df[['tierRank', 'position', 'name', 'result']]

In [7]:
data

,tierRank,position,name,result
0,P318,S,럭스,WIN
1,P466,S,샤코,LOSE
2,P329,M,아트록스,WIN
3,P461,T,리븐,WIN
4,P30,A,애쉬,WIN
...,...,...,...,...
999995,S146,S,블리츠크랭크,WIN
999996,G442,M,아크샨,WIN
999997,G418,J,리 신,LOSE
999998,S178,A,이즈리얼,WIN


In [8]:
# 결측치 드랍
data = data.dropna(axis=0)
# 다시하기 드랍
data.drop(data.loc[data['result']=='UNKNOWN'].index, inplace=True)
print('data : ', len(data))

data :  963577


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [9]:
# result 숫자로 변환
data.loc[data['result'] == 'WIN', 'result'] = 1
data.loc[data['result'] == 'LOSE', 'result'] = 0
data['game'] = 1
data = data.astype({'result' : 'int'})

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
<ipython-input-9-6d31ce203f59>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

In [10]:
# 티어별 데이터 프레임 생성
IRON = data[data['tierRank'].str.contains('I')]
print('IRON : ', len(IRON))
BRONZE = data[data['tierRank'].str.contains('B')]
print('BRONZE : ', len(BRONZE))
SILVER = data[data['tierRank'].str.contains('S')]
print('SILVER : ', len(SILVER))
GOLD = data[data['tierRank'].str.contains('G')]
print('GOLD : ', len(GOLD))
PLATINUM = data[data['tierRank'].str.contains('P')]
print('PLATINUM : ', len(PLATINUM))
DIAMOND = data[data['tierRank'].str.contains('D')]
print('DIAMOND : ', len(DIAMOND))
MASTER = data[data['tierRank'].str.contains('M')]
print('MASTER : ', len(MASTER))
GRAND_MASTER = data[data['tierRank'].str.contains('R')]
print('GRAND_MASTER : ', len(GRAND_MASTER))
CHALLENGER = data[data['tierRank'].str.contains('C')]
print('CHALLENGER : ', len(CHALLENGER))
print('SUM : ', len(IRON)+len(BRONZE)+len(SILVER)+len(GOLD)+len(PLATINUM)+len(DIAMOND)+len(MASTER)+len(GRAND_MASTER)+len(CHALLENGER))

IRON :  9857
BRONZE :  125597
SILVER :  344653
GOLD :  328557
PLATINUM :  124452
DIAMOND :  26585
MASTER :  2953
GRAND_MASTER :  712
CHALLENGER :  211
SUM :  963577


In [11]:
IRON_group = IRON.groupby(['position', 'name'], as_index=False).sum()
IRON_group['winRate'] = IRON_group['result'] / IRON_group['game']
IRON_group = IRON_group.sort_values(by=['position', 'winRate'], ascending=False) 

BRONZE_group = BRONZE.groupby(['position', 'name'], as_index=False).sum()
BRONZE_group['winRate'] = BRONZE_group['result'] / BRONZE_group['game']
BRONZE_group = BRONZE_group.sort_values(by=['position', 'winRate'], ascending=False) 

SILVER_group = SILVER.groupby(['position', 'name'], as_index=False).sum()
SILVER_group['winRate'] = SILVER_group['result'] / SILVER_group['game']
SILVER_group = SILVER_group.sort_values(by=['position', 'winRate'], ascending=False) 

GOLD_group = GOLD.groupby(['position', 'name'], as_index=False).sum()
GOLD_group['winRate'] = GOLD_group['result'] / GOLD_group['game']
GOLD_group = GOLD_group.sort_values(by=['position', 'winRate'], ascending=False) 

PLATINUM_group = PLATINUM.groupby(['position', 'name'], as_index=False).sum()
PLATINUM_group['winRate'] = PLATINUM_group['result'] / PLATINUM_group['game']
PLATINUM_group = PLATINUM_group.sort_values(by=['position', 'winRate'], ascending=False) 

DIAMOND_group = DIAMOND.groupby(['position', 'name'], as_index=False).sum()
DIAMOND_group['winRate'] = DIAMOND_group['result'] / DIAMOND_group['game']
DIAMOND_group = DIAMOND_group.sort_values(by=['position', 'winRate'], ascending=False) 

MASTER_group = MASTER.groupby(['position', 'name'], as_index=False).sum()
MASTER_group['winRate'] = MASTER_group['result'] / MASTER_group['game']
MASTER_group = MASTER_group.sort_values(by=['position', 'winRate'], ascending=False) 

GRAND_MASTER_group = GRAND_MASTER.groupby(['position', 'name'], as_index=False).sum()
GRAND_MASTER_group['winRate'] = GRAND_MASTER_group['result'] / GRAND_MASTER_group['game']
GRAND_MASTER_group = GRAND_MASTER_group.sort_values(by=['position', 'winRate'], ascending=False) 

CHALLENGER_group = CHALLENGER.groupby(['position', 'name'], as_index=False).sum()
CHALLENGER_group['winRate'] = CHALLENGER_group['result'] / CHALLENGER_group['game']
CHALLENGER_group = CHALLENGER_group.sort_values(by=['position', 'winRate'], ascending=False) 

In [18]:
IRON_group

,position,name,result,game,winRate
391,T,니달리,1,1,1.000000
394,T,다이애나,1,1,1.000000
405,T,리산드라,2,2,1.000000
416,T,벨코즈,1,1,1.000000
418,T,브라움,1,1,1.000000
419,T,브랜드,1,1,1.000000
422,T,뽀삐,4,4,1.000000
423,T,사이온,1,1,1.000000
426,T,세나,1,1,1.000000
430,T,쉬바나,2,2,1.000000


In [12]:
# # 시각화
# plt.rc('font', family='Malgun Gothic')

# fig, axes = plt.subplots(nrows=5
#                          , ncols=1
#                          , sharex=True
#                          , figsize=(10,15))

# position = ['T', 'M', 'A', 'S', 'J']

# for i, ax in enumerate(axes):
#     DIAMOND_group_i = DIAMOND_group[IRON_group['position'] == position[i]]
#     DIAMOND_group_i = DIAMOND_group_i[0:21]
#     ax.barh(DIAMOND_group_i['name'], DIAMOND_group_i['winRate'], color='blue', alpha=0.5)

# plt.tight_layout()

In [13]:
pd.set_option('display.max_rows', None)
DIAMOND_group

,position,name,result,game,winRate
362,T,니달리,1,1,1.000000
371,T,렉사이,1,1,1.000000
399,T,세라핀,1,1,1.000000
407,T,신드라,3,3,1.000000
412,T,아지르,3,3,1.000000
439,T,조이,1,1,1.000000
440,T,직스,1,1,1.000000
446,T,카서스,1,1,1.000000
449,T,칼리스타,3,3,1.000000
455,T,키아나,1,1,1.000000
